tf.distributed.strategy : 훈련을 여러 처리 장치들로 분산시키는 것을 추상화함

tf.distributed.MirroredStrategy : 동기화된 훈련 방식을 활용하여 한 장비에 있는 여러 개의 GPU로 그래프 내 복제를 수행 -> 각 프로세서의 gradient를 all-reduce를 사용하여 모음 -> 모아서 계산한 값을 각 프로세서의 모델 복사본에 적용

In [31]:
!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

     |████████████████████████████████| 380.5 MB 9.7 kB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 4.3 MB 49.5 MB/s 
     |████████████████████████████████| 501 kB 47.4 MB/s 


In [32]:
datasets, info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

mnist_train, mnist_test = datasets['train'], datasets['test']

In [33]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [34]:
print('장치의 수 : {}'.format(strategy.num_replicas_in_sync))

장치의 수 : 1


### 입력 파이프라인 구성하기

다중 GPU로 모델을 훈련할 때는 배치 크기를 늘려야 컴퓨팅 자원을 효과적으로 사용 가능함
GPU 메모리에 맞추어 가능한 가장 큰 배치 크기 사용 -> 학습률 조정

In [35]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [36]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [37]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [38]:
with strategy.scope():
  model = tf.keras.Sequential([
                               tf.keras.layers.Conv2D(32, 3, activation = 'relu', input_shape = (28,28,1)),
                               tf.keras.layers.MaxPooling2D(),
                               tf.keras.layers.Flatten(),
                               tf.keras.layers.Dense(64, activation = 'relu'),
                               tf.keras.layers.Dense(10, activation = 'softmax')
  ])

  model.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

### 콜백 정의하기

tensorboard : 텐서보드용 로그를 남겨 텐서보드에서 그래프를 그리게 함
model checkpoint : 매 에포크가 끝난 후 모델을 저장
learning rate scheduler : 매 에포크 혹은 배치가 끝난 후 학습률을 바꿀 수 있음

In [39]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

In [40]:
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [41]:
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = None):
    print('\n 에포크 {}의 학습률은 {}입니다.'.format(epoch + 1 , model.optimizer.lr.numpy()))

In [42]:
callbacks = [
             tf.keras.callbacks.TensorBoard(log_dir = './dirs'),
             tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only = True),
             tf.keras.callbacks.LearningRateScheduler(decay),
             PrintLR()
]

훈련 & 평가

In [43]:
model.fit(train_dataset, epochs = 12, callbacks = callbacks)

Epoch 1/12
  3/938 [..............................] - ETA: 3:53 - loss: 2.2092 - accuracy: 0.2396WARNING:tensorflow:Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0091s vs `on_train_batch_begin` time: 0.0287s). Check your callbacks.


938/938 [==============================] - 19s 16ms/step - loss: 0.2131 - accuracy: 0.9381

 에포크 1의 학습률은 0.0010000000474974513입니다.
Epoch 2/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0670 - accuracy: 0.9799

 에포크 2의 학습률은 0.0010000000474974513입니다.
Epoch 3/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0458 - accuracy: 0.9861

 에포크 3의 학습률은 0.0010000000474974513입니다.
Epoch 4/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0253 - accuracy: 0.9930

 에포크 4의 학습률은 9.999999747378752e-05입니다.
Epoch 5/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0223 - accuracy: 0.9941

 에포크 5의 학습률은 9.999999747378752e-05입니다.
Epoch 6/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0207 - accuracy: 0.9944

 에포크 6의 학습률은 9.999999747378752e-05입니다.
Epoch 7/12
938/938 [==============================] - 9s 9ms/step - loss: 0.0191 - accuracy: 0.9952

 에포크 7의 학습률은 9.999999747378752e-05입니다.
Epoch 8/12
938/938 [=========

In [44]:
ls {checkpoint_dir}

checkpoint                   ckpt_4.data-00000-of-00001
ckpt_10.data-00000-of-00001  ckpt_4.index
ckpt_10.index                ckpt_5.data-00000-of-00001
ckpt_11.data-00000-of-00001  ckpt_5.index
ckpt_11.index                ckpt_6.data-00000-of-00001
ckpt_12.data-00000-of-00001  ckpt_6.index
ckpt_12.index                ckpt_7.data-00000-of-00001
ckpt_1.data-00000-of-00001   ckpt_7.index
ckpt_1.index                 ckpt_8.data-00000-of-00001
ckpt_2.data-00000-of-00001   ckpt_8.index
ckpt_2.index                 ckpt_9.data-00000-of-00001
ckpt_3.data-00000-of-00001   ckpt_9.index
ckpt_3.index


In [45]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
eval_loss, eval_acc = model.evaluate(eval_dataset)
print('평가 손실 : {}, 평가 정확도 : {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 4s 15ms/step - loss: 0.0391 - accuracy: 0.9870
평가 손실 : 0.03905589133501053, 평가 정확도 : 0.9869999885559082


In [46]:
import tensorboard

In [47]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


SavedModel로 내보내기

In [49]:
path = 'saved_model/'

In [51]:
model.save(path)

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [53]:
# strategy.scope 없이 모델 부르기

unreplicated_model = tf.keras.models.load_model(path)

In [54]:
unreplicated_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('평가 손실: {}, 평가 정확도: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 1s 6ms/step - loss: 0.0391 - accuracy: 0.9870
평가 손실: 0.03905589133501053, 평가 정확도: 0.9869999885559082


In [55]:
# strategy.scope으로 부르기

with strategy.scope():
  replicated_model = tf.keras.models.load_model(path)
  replicated_model.compile(loss='sparse_categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(),
                           metrics = ['accuracy'])
  eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
  print('평가 손실: {}, 평가 정확도: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 4s 8ms/step - loss: 0.0391 - accuracy: 0.9870
평가 손실: 0.03905589133501053, 평가 정확도: 0.9869999885559082
